In [27]:
import os
import pg8000
import pandas as pd
from pandas import ExcelWriter
import re

In [2]:
#code from Agata
#these are nice functions to open LIMS, make a query and then close LIMS after

def _connect(user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    conn = pg8000.connect(user=user, host=host, database=database, password=password, port=port)
    return conn, conn.cursor()

def _select(cursor, query):
    cursor.execute(query)
    columns = [ d[0] for d in cursor.description ]
    return [ dict(zip(columns, c)) for c in cursor.fetchall() ]

def limsquery(query, user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    """A function that takes a string containing a SQL query, connects to the LIMS database and outputs the result."""
    conn, cursor = _connect(user, host, database, password, port)
    try:
        results = _select(cursor, query)
    finally:
        
        #THESE ARE IMPORTANT!!!!!!
        #Every query needs to be closed when done
        cursor.close()             
        conn.close()
    return results

#this last function will take our query results and put them in a dataframe so that they are easy to work with
def get_lims_dataframe(query):
    '''Return a dataframe with lims query'''
    result = limsquery(query)
    try:
        data_df = pd.DataFrame(data=result, columns=result[0].keys())
    except IndexError:
        print "Could not find results for your query."
        data_df = pd.DataFrame()
    return data_df

# Make user report

In [3]:
def user_df(P_number):
    """Saves an excel dataframe with all patched cells for a specified user in current directory.
     
    Parameters
    ----------
    P_number : A string. Corresponds to the user's P number in the form of P1, P2, P3, etc.
       
    Returns
    -------
    None
    """
    user = master_df[master_df["patched_cell_container"].str.contains(P_number)]
    user.sort_values("patched_cell_container", axis = 0, inplace = True)
    user = user.loc[:,['name', 'patched_cell_container','date', 'rna_qc', 'go_no_go_63x', 'link_20x', 'dendrite_type']]
    writer = pd.ExcelWriter('user_dataframe.xlsx')
    user.to_excel(writer,'Sheet1')

In [4]:
def cell_count(P_number, df):
    """Return the number of cells in a specified dataframe for a specified user.
     
    Parameters
    ----------
    P_number : A string. Corresponds to the user's P number in the form of P1, P2, P3, etc.
    df: a pandas dataframe
    
    Returns
    -------
    An integer cell number
    """
    user = df[df["patched_cell_container"].str.contains(P_number)]
    return user["patched_cell_container"].count()

In [5]:
def gen_filter(df, col, cond):
    filtered = df[df[col] == cond]
    return filtered

In [35]:
def per_user(P_number):
    """Returns a summary of patcher metrics for a specified user.
     
    Parameters
    ----------
    P_number : A string. Corresponds to the user's P number in the form of P1, P2, P3, etc.
        
    Returns
    -------
    Returns None
    Prints a summary of patcher metrics, including total cells patched, cells that passed RNA seq QC, and QC pass percentage
    """
    total_cells = cell_count(P_number,master_df)
    cells_passed_qc = cell_count(P_number,qcpass)
    qc_pass_percentage = float((cell_count(P_number,qcpass)))/float((cell_count(P_number,master_df)))
    mouse_cells = cell_count(P_number, mouse)
    human_cells = cell_count(P_number, human)
    imaged_cells = cell_count(P_number,imagego)
    return (total_cells, cells_passed_qc, qc_pass_percentage, mouse_cells, human_cells, imaged_cells)

In [38]:
query = """
WITH do_63x(cell_id, go) AS (SELECT DISTINCT cell.id, array_to_string(array_agg(DISTINCT tag.name), ' _AND_ ') 
FROM specimens cell JOIN ephys_roi_results err ON err.id = cell.ephys_roi_result_id 
JOIN specimen_tags_specimens sptagsp ON sptagsp.specimen_id = cell.id 
JOIN specimen_tags tag ON tag.id = sptagsp.specimen_tag_id AND tag.id in (602120185,602122082) 
GROUP BY cell.id ORDER BY 1), 
dendrite_type AS (SELECT sts.specimen_id,array_to_string(array_agg(DISTINCT tag.name), ' ') AS dendrite_type \
FROM specimen_tags_specimens sts JOIN specimen_tags tag on sts.specimen_tag_id = tag.id \
WHERE tag.name in('dendrite type - spiny', 'dendrite type - NA', 'dendrite type - sparsely spiny', 'dendrite type - aspiny') \
GROUP BY sts.specimen_id),
imgs20 AS (WITH tag_list AS (SELECT DISTINCT ims.id AS ims_id, imst.name AS tag FROM image_series ims 
JOIN image_series_image_series_tags ims2imst ON ims2imst.image_series_id = ims.id 
JOIN image_series_tags imst ON imst.id = ims2imst.image_series_tag_id 
ORDER BY 1,2) 
SELECT DISTINCT ims.id AS image_series_id, ims.workflow_state AS workflow_state, ims.specimen_id, 
ARRAY_TO_STRING(ARRAY_AGG(DISTINCT tag_list.tag), '_AND_') AS tags FROM image_series ims 
LEFT JOIN tag_list ON tag_list.ims_id = ims.id 
WHERE ims.type = 'FocalPlaneImageSeries' AND is_stack = false 
GROUP BY ims.id, ims.workflow_state, ims.specimen_id)
SELECT DISTINCT s.patched_cell_container, s.created_at AS date, do_63x.go AS go_no_go_63x, CASE WHEN ra.id IS NOT NULL THEN (CASE WHEN ra.failed = 't' THEN 'failed' ELSE 'passed' END) ELSE '' END AS rna_qc, s.name,
'=HYPERLINK("http://lims2/focal_plane_image_series?id=' || imgs20.image_series_id || '")' as link_20x, dendrite_type
FROM specimens s 
LEFT JOIN projects proj ON s.project_id = proj.id 
LEFT JOIN do_63x ON do_63x.cell_id = s.id 
JOIN specimens slice ON s.parent_id = slice.id
LEFT JOIN rna_amplification_inputs rai on rai.sample_id = s.id 
LEFT JOIN imgs20 ON slice.id = imgs20.specimen_id 
LEFT JOIN rna_amplifications ra on ra.id = rai.rna_amplification_id 
LEFT JOIN dendrite_type on s.id = dendrite_type.specimen_id
WHERE proj.code <> 'mMPATCH' AND 
s.patched_cell_container IS NOT NULL 
AND s.created_at > '2017-10-01'
"""


#AND (err.recording_date > '2017-10-01' OR err.recording_date IS NULL)
#AND err.recording_date > '2017-10-01'

master_df = get_lims_dataframe(query)
master_df

,go_no_go_63x,name,rna_qc,dendrite_type,patched_cell_container,link_20x,date
0,63x go,Ctgf-T2A-dgCre;Ai14-374416.07.01.01,passed,None,P9S4_180209_404_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-02-09 22:25:31.098048
1,None,H17.06.015.13.09.02,failed,None,PAS4_171205_453_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2017-12-06 03:51:20.666953
2,None,Ntsr1-Cre_GN220;Ai14-361815.05.02.01,failed,None,PAS4_171215_451_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2017-12-15 18:40:32.255491
3,63x no go,Oxtr-T2A-Cre;Ai14-351471.03.02.03,failed,dendrite type - NA,P2S4_171004_053_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2017-10-04 17:49:59.361188
4,63x no go,Htr3a-Cre_NO152;Ai14-368863.04.02.01,passed,dendrite type - aspiny,P8S4_180124_356_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-01-25 01:20:32.771639
5,None,Slc17a8-iCre;Ai14-387899.04.01.02,,None,P9S4_180511_401_A01,None,2018-05-11 18:15:45.880427
6,None,H18.06.001.11.11.03,failed,None,P8S4_180201_355_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-02-02 01:45:39.921595
7,None,Vip-IRES-Cre;Ai14-373812.03.02.02,passed,None,P2S4_180205_054_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-02-05 23:10:35.524189
8,None,H17.03.014.11.04.02,passed,None,PBS4_171010_508_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2017-10-11 03:39:37.330273
9,None,H17.26.003.11.15.01,failed,None,P6S4_171128_253_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2017-11-28 22:55:33.833615


In [8]:
qcpass = gen_filter(master_df,'rna_qc', 'passed')
qcpass

,go_no_go_63x,name,rna_qc,dendrite_type,patched_cell_container,link_20x,date
0,63x go,Ctgf-T2A-dgCre;Ai14-374416.07.01.01,passed,None,P9S4_180209_404_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-02-09 22:25:31.098048
2,63x no go,Htr3a-Cre_NO152;Ai14-368863.04.02.01,passed,dendrite type - aspiny,P8S4_180124_356_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-01-25 01:20:32.771639
6,None,Vip-IRES-Cre;Ai14-373812.03.02.02,passed,None,P2S4_180205_054_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-02-05 23:10:35.524189
7,None,H17.03.014.11.04.02,passed,None,PBS4_171010_508_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2017-10-11 03:39:37.330273
8,63x no go,H18.06.001.11.11.01,passed,dendrite type - NA,P8S4_180201_353_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-02-02 00:30:39.993504
9,None,Slc32a1-IRES-Cre;Ai14-379844.03.02.02,passed,None,P2S4_180314_054_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-03-14 22:30:35.952233
10,63x no go,Ndnf-IRES2-dgCre;Ai14-377310.05.02.01,passed,dendrite type - NA,PAS4_180227_453_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-02-27 20:05:34.682013
11,None,Slc17a6-IRES-Cre;Ai14-386824.04.02.01,passed,None,P8S4_180502_358_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-05-02 22:10:38.177565
12,None,Sst-IRES-Cre;Ai14-386474.04.02.02,passed,None,P8S4_180423_352_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-04-23 18:15:38.724590
13,63x no go,Rbp4-Cre_KL100;Ai14-366313.04.01.01,passed,dendrite type - NA,P9S4_171211_406_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2017-12-11 23:05:33.403064


In [9]:
imagego = gen_filter(master_df,'go_no_go_63x', '63x go')
imagego

,go_no_go_63x,name,rna_qc,dendrite_type,patched_cell_container,link_20x,date
0,63x go,Ctgf-T2A-dgCre;Ai14-374416.07.01.01,passed,None,P9S4_180209_404_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-02-09 22:25:31.098048
20,63x go,Crh-IRES-Cre_ZJH;Sst-IRES-FlpO;Ai65-362355.04....,passed,None,P8S4_171129_353_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2017-11-29 19:10:31.796143
21,63x go,H17.03.016.11.09.06,passed,None,P8S4_171220_356_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2017-12-20 23:40:40.274418
23,63x go,Slc32a1-IRES2-FlpO;Ai65F-363946.05.02.01,passed,None,PAS4_171219_451_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2017-12-19 18:35:31.986715
34,63x go,Chat-IRES-Cre-neo;Ai14-373623.04.02.02,passed,None,PBS4_180205_505_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-02-05 23:10:43.527655
36,63x go,Sst-IRES-Cre;Ai14-381578.04.01.01,passed,None,P1S4_180319_001_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-03-19 17:50:36.913026
43,63x go,Chrna2-Cre_OE25;Ai14-367171.04.02.03,passed,None,P2S4_180110_053_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-01-10 20:05:37.274293
46,63x go,Ndnf-IRES2-dgCre;Slc32a1-IRES2-FlpO;Ai65-36876...,passed,None,PAS4_180123_451_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-01-23 18:35:33.272676
52,63x go,Tac1-IRES2-Cre;Sst-IRES-FlpO;Ai65-360614.03.01.01,passed,None,P9S4_171201_408_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2017-12-01 22:50:32.452735
54,63x go,Ndnf-IRES2-dgCre;Slc32a1-IRES2-FlpO;Ai65-37811...,passed,None,P1S4_180305_001_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-03-05 19:10:40.284555


In [34]:
master_df['human?'] = master_df['name'].str.match(r"H\d\d") #creates new boolean column
human = gen_filter(master_df, 'human?', True)
mouse = gen_filter(master_df, 'human?', False)
mouse

,go_no_go_63x,name,rna_qc,dendrite_type,patched_cell_container,link_20x,date,human?
0,63x go,Ctgf-T2A-dgCre;Ai14-374416.07.01.01,passed,None,P9S4_180209_404_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-02-09 22:25:31.098048,False
2,63x no go,Htr3a-Cre_NO152;Ai14-368863.04.02.01,passed,dendrite type - aspiny,P8S4_180124_356_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-01-25 01:20:32.771639,False
3,63x no go,Oxtr-T2A-Cre;Ai14-351471.03.02.03,failed,dendrite type - NA,P2S4_171004_053_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2017-10-04 17:49:59.361188,False
4,None,Slc17a8-iCre;Ai14-387899.04.01.02,,None,P9S4_180511_401_A01,None,2018-05-11 18:15:45.880427,False
6,None,Vip-IRES-Cre;Ai14-373812.03.02.02,passed,None,P2S4_180205_054_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-02-05 23:10:35.524189,False
9,None,Slc32a1-IRES-Cre;Ai14-379844.03.02.02,passed,None,P2S4_180314_054_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-03-14 22:30:35.952233,False
10,63x no go,Ndnf-IRES2-dgCre;Ai14-377310.05.02.01,passed,dendrite type - NA,PAS4_180227_453_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-02-27 20:05:34.682013,False
11,None,Slc17a6-IRES-Cre;Ai14-386824.04.02.01,passed,None,P8S4_180502_358_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-05-02 22:10:38.177565,False
12,None,Sst-IRES-Cre;Ai14-386474.04.02.02,passed,None,P8S4_180423_352_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-04-23 18:15:38.724590,False
13,63x no go,Rbp4-Cre_KL100;Ai14-366313.04.01.01,passed,dendrite type - NA,P9S4_171211_406_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2017-12-11 23:05:33.403064,False


In [10]:
per_user('P8')

724

In [22]:
user_df('PA')

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [11]:
def validated_input(prompt_text, invalid_response, valid_options=None):
    """Keep asking user for input until a valid input has been entered"""
    while True:
        result = raw_input(prompt_text)
        if (valid_options) and (result not in valid_options):
            response = invalid_response
            print response
            continue
        else:
            break
    return result

In [36]:
def main():
    """Prompts the user about the operator on which they'd like to report, 
    then prints info about that operator's patching statistics.
    """

    #Ask for user input
    str_prompt = "\nOn what operator would you like to report? (P#): "
    valid_vals = ["P1", "P2", "P8", "P9", "PA", "PB"]

    response = "\nPlease try again...rig operator should be in the form P#"

    P_number = validated_input(str_prompt, response, valid_vals)
    
    [total_cells, cells_passed_qc, qc_pass_percentage, mouse_cells, human_cells, imaged_cells] = per_user(P_number)
    print "Total cells patched: %d" %total_cells
    print "Cells that passed RNA seq QC: %d" %cells_passed_qc
    print "QC pass percentage: %f" %qc_pass_percentage
    print "Mouse cells: %d" %mouse_cells
    print "Human cells: %d" %human_cells
    print "Cells imaged at 63x: %d" %imaged_cells
    
    user_df(P_number)

In [40]:
main()


On what operator would you like to report? (P#): P2
Total cells patched: 455
Cells that passed RNA seq QC: 284
QC pass percentage: 0.624176
Mouse cells: 331
Human cells: 59
Cells imaged at 63x: 98


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [43]:
search = "SELECT * FROM specimen_tags"
search_result = limsquery(search)
first_element = search_result[0]
print first_element.keys()

['description', 'created_at', 'updated_at', 'is_public', 'id', 'name']


In [46]:
search = "SELECT sts.specimen_id,array_to_string(array_agg(DISTINCT tag.name), ' ') AS dendrite_type \
FROM specimen_tags_specimens sts JOIN specimen_tags tag on sts.specimen_tag_id = tag.id \
WHERE tag.name in('dendrite type - spiny', 'dendrite type - NA', 'dendrite type - sparsely spiny', 'dendrite type - aspiny') \
GROUP BY sts.specimen_id"
lims_df = get_lims_dataframe(search)
lims_df.head()

,specimen_id,dendrite_type
0,309721694.0,dendrite type - spiny
1,311853612.0,dendrite type - spiny
2,312171141.0,dendrite type - spiny
3,312171887.0,dendrite type - spiny
4,312409447.0,dendrite type - spiny
